In [1]:
import nltk
import string
import spacy
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel
import warnings
from tqdm._tqdm_notebook import tqdm_notebook
from scipy.spatial import distance
import numpy as np
import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
from sentence_transformers import SentenceTransformer
from nltk.corpus import stopwords
import tensorflow as tf
import transformers
from sent2vec.vectorizer import Vectorizer as S2vectorizer
tqdm_notebook.pandas()
warnings.filterwarnings("ignore")

pd.set_option('display.max_colwidth', 0)
pd.set_option('display.max_columns', 0)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  


ModuleNotFoundError: No module named 'sentence_transformers'

In [5]:
df = pd.read_csv('../data/SQuAD_csv.csv', encoding='utf-8').loc[:, ['context']]
df = df.drop_duplicates(subset=['context'])
df['context'] = df['context'].apply(lambda x: str(x).lower())

In [6]:
stop_words = stopwords.words("english")
def clean(text):
    text = ' '.join([word for word in text.split() if word not in stop_words])
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\s{2,}', " ", text)
    return text

In [7]:
df['cleaned_context'] = df['context'].apply(clean)

In [8]:
df.head(3)

,context,cleaned_context
0,"beyoncé giselle knowles-carter (/biːˈjɒnseɪ/ bee-yon-say) (born september 4, 1981) is an american singer, songwriter, record producer and actress. born and raised in houston, texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of r&b girl-group destiny's child. managed by her father, mathew knowles, the group became one of the world's best-selling girl groups of all time. their hiatus saw the release of beyoncé's debut album, dangerously in love (2003), which established her as a solo artist worldwide, earned five grammy awards and featured the billboard hot 100 number-one singles ""crazy in love"" and ""baby boy"".",beyoncé giselle knowles carter biːˈjɒnseɪ bee yon say born september 4 1981 american singer songwriter record producer actress born raised houston texas performed various singing dancing competitions child rose fame late 1990s lead singer r b girl group destiny s child managed father mathew knowles group became one world s best selling girl groups time hiatus saw release beyoncé s debut album dangerously love 2003 established solo artist worldwide earned five grammy awards featured billboard hot 100 number one singles crazy love baby boy
15,"following the disbandment of destiny's child in june 2005, she released her second solo album, b'day (2006), which contained hits ""déjà vu"", ""irreplaceable"", and ""beautiful liar"". beyoncé also ventured into acting, with a golden globe-nominated performance in dreamgirls (2006), and starring roles in the pink panther (2006) and obsessed (2009). her marriage to rapper jay z and portrayal of etta james in cadillac records (2008) influenced her third album, i am... sasha fierce (2008), which saw the birth of her alter-ego sasha fierce and earned a record-setting six grammy awards in 2010, including song of the year for ""single ladies (put a ring on it)"". beyoncé took a hiatus from music in 2010 and took over management of her career; her fourth album 4 (2011) was subsequently mellower in tone, exploring 1970s funk, 1980s pop, and 1990s soul. her critically acclaimed fifth studio album, beyoncé (2013), was distinguished from previous releases by its experimental production and exploration of darker themes.",following disbandment destiny s child june 2005 released second solo album b day 2006 contained hits déjà vu irreplaceable beautiful liar beyoncé also ventured acting golden globe nominated performance dreamgirls 2006 starring roles pink panther 2006 obsessed 2009 marriage rapper jay z portrayal etta james cadillac records 2008 influenced third album am sasha fierce 2008 saw birth alter ego sasha fierce earned record setting six grammy awards 2010 including song year single ladies put ring it beyoncé took hiatus music 2010 took management career fourth album 4 2011 subsequently mellower tone exploring 1970s funk 1980s pop 1990s soul critically acclaimed fifth studio album beyoncé 2013 distinguished previous releases experimental production exploration darker themes
27,"a self-described ""modern-day feminist"", beyoncé creates songs that are often characterized by themes of love, relationships, and monogamy, as well as female sexuality and empowerment. on stage, her dynamic, highly choreographed performances have led to critics hailing her as one of the best entertainers in contemporary popular music. throughout a career spanning 19 years, she has sold over 118 million records as a solo artist, and a further 60 million with destiny's child, making her one of the best-selling music artists of all time. she has won 20 grammy awards and is the most nominated woman in the award's history. the recording industry association of america recognized her as the top certified artist in america during the 2000s decade. in 2009, billboard named her the top radio songs artist of the decade, the top female artist of the 2000s and their artist of the millennium in 2011. time listed her 

In [9]:
vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform(df['cleaned_context'])

In [10]:
print(tfidf.shape)

(18877, 78498)


In [21]:
def get_similar_doc(tfidfvectorizer, docs_tfidf, query):
    """
    vectorizer: TfIdfVectorizer model
    docs_tfidf: tfidf vectors for all docs
    query: query doc

    return: doc with highest tf-idf cosine similarity
    """
    
    query_tfidf = tfidfvectorizer.transform([query])
    cosineSimilarities = cosine_similarity(query_tfidf, docs_tfidf).flatten()
    cosineSimilarities = np.array(cosineSimilarities)
    return cosineSimilarities.argsort()[-3:][::-1]

def get_relevant_sentence(doc_ids, query):
    docs = df.iloc[doc_ids, df.columns.get_loc('context')].apply(lambda x: [i.strip() for i in x.split('.') if len(i)>1])
    sentences = []
    for doc in docs:
        for sentence in doc:
            sentences.append(sentence)

    vectorizer = S2vectorizer()
    vectorizer.bert([query] + sentences)
    vectors = vectorizer.vectors
    query_vec, other_vec = vectors[0], vectors[1:]
    
    print("----- Sentences Retrieved: -----")
    for idx, sentence in enumerate(sentences):
        print(f"{idx}. {sentence}")
    
    distances = distance.cdist([query_vec], other_vec, "cosine")[0]
    min_index = np.argmin(distances)
#     min_distance = distances[min_index]
#     max_similarity = 1 - min_distance
    return sentences[min_index]
    
def get_answer(query):
    query = re.sub('[%s]' % re.escape(string.punctuation), '', query)
    query = re.sub('\s{2,}', " ", query).lower()
    most_similar_doc_ids = get_similar_doc(vectorizer, tfidf, query)
    return get_relevant_sentence(most_similar_doc_ids, query)

In [219]:
get_answer("when did the group release their multi-platinum second album The Writing's on the Wall?")

----- Sentences Retrieved: -----
0. in july 1973, queen finally under a trident/emi deal released their eponymous debut album, an effort influenced by the heavy metal and progressive rock of the day
1. the album was received well by critics; gordon fletcher of rolling stone said "their debut album is superb", and chicago's daily herald called it an "above average debut"
2. it drew little mainstream attention, and the lead single "keep yourself alive", a brian may composition, sold poorly
3. retrospectively, "keep yourself alive" is cited as the highlight of the album, and in 2008 rolling stone ranked it 31st in the "100 greatest guitar songs of all time", describing it as "an entire album's worth of riffs crammed into a single song"
4. the album was certified gold in the uk and the us
5. beginning his second effort that fall, west would invest two million dollars and take over a year to craft his second album
6. west was significantly inspired by roseland nyc live, a 1998 live album by

'once he had completed the album, it was leaked months before its release date'

## ^ Results not good, I'm thinking can try split into sentence first before tf-idf. Or if there's some better way to find similarity between a short query and long paragraphs?

## Also feel like clause extraction may make things even worse cuz it doesn't retain the nouns and stuff after splitting into clauses

In [64]:
def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

def get_similar_doc(tfidfvectorizer, docs_tfidf, query):
    """
    vectorizer: TfIdfVectorizer model
    docs_tfidf: tfidf vectors for all docs
    query: query doc

    return: doc with highest tf-idf cosine similarity
    """
    
    query_tfidf = tfidfvectorizer.transform([query])
    cosineSimilarities = cosine_similarity(query_tfidf, docs_tfidf).flatten()
    cosineSimilarities = np.array(cosineSimilarities)
    return cosineSimilarities.argsort()[-3:][::-1]

def get_sentences(doc_ids):
    docs = df.iloc[doc_ids, df.columns.get_loc('context')].apply(lambda x: [i.strip() for i in x.split('.') if len(i)>1])
    print(docs)
    sentences = []
    for doc in docs:
        for sentence in doc:
            sentences.append(sentence)
    return sentences

def infersent(sentences, query_vec):
    similarity = []
    for sent in sentences:
        sim = cosine(query_vec, model.encode([sent])[0])
        similarity.append((sent, sim))

    return similarity
# .sort(key=lambda x: x[1], reverse=True)

    
def get_answer(query):
    query = re.sub('[%s]' % re.escape(string.punctuation), '', query)
    query = re.sub('\s{2,}', " ", query).lower()
    most_similar_doc_ids = get_similar_doc(vectorizer, tfidf, query)
    sentences = get_sentences(most_similar_doc_ids)
    model.build_vocab(sentences, tokenize=True)
    query_vec = model.encode(query)[0]
    
    answers = infersent(sentences, query_vec)
#     print(sorted(answers, key=lambda x: x[1]))
    return sorted(answers, key=lambda x: x[1], reverse=True)


In [11]:
! mkdir encoder
! curl -Lo encoder/infersent2.pkl https://dl.fbaipublicfiles.com/infersent/infersent2.pkl
  
! mkdir GloVe
! curl -Lo GloVe/glove.840B.300d.zip http://nlp.stanford.edu/data/glove.840B.300d.zip
! unzip GloVe/glove.840B.300d.zip -d GloVe/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  146M  100  146M    0     0  9193k      0  0:00:16  0:00:16 --:--:-- 10.6M3k      0  0:00:17  0:00:13  0:00:04 10.5M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0   315    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0   352    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100 2075M  100 2075M    0     0  4099k      0  0:08:38  0:08:38 --:--:-- 4491k   0  0:12:38  0:00:14  0:12:24 4925k 0:07:21 4889k4489k      0  0:07:53  0:00:50  0:07:03 5368k      0  0:07:30  0:01:19  0:06:11 5457k:27  0:01:27  0:06:00 5168k0k      0  0:07:26  0:01:31  0:05:55 5069k393k     0  0:07:06  0:03:53  0:03:13 4942k  3879k      0  0:09:07  0:05:10  0:03:57  100k 0:03:51 5159k38  0:08:23  0:00:

In [19]:
from models import InferSent
import torch

V = 2
MODEL_PATH = 'encoder/infersent%s.pkl' % V
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': V}
model = InferSent(params_model)
model.load_state_dict(torch.load(MODEL_PATH))

W2V_PATH = 'GloVe/glove.840B.300d.txt'
model.set_w2v_path(W2V_PATH)

In [65]:
get_answer("when did the group release their multi-platinum second album The Writing's on the Wall?")

41419    [in july 1973, queen finally under a trident/emi deal released their eponymous debut album, an effort influenced by the heavy metal and progressive rock of the day, the album was received well by critics; gordon fletcher of rolling stone said "their debut album is superb", and chicago's daily herald called it an "above average debut", it drew little mainstream attention, and the lead single "keep yourself alive", a brian may composition, sold poorly, retrospectively, "keep yourself alive" is cited as the highlight of the album, and in 2008 rolling stone ranked it 31st in the "100 greatest guitar songs of all time", describing it as "an entire album's worth of riffs crammed into a single song", the album was certified gold in the uk and the us]                                                                                                                                                                                                                                              

[('late registration sold over 2', 0.32071656),
 ('the album was certified gold in the uk and the us', 0.11131949),
 ('once he had completed the album, it was leaked months before its release date',
  0.10193596),
 ('he recorded the remainder of the album in los angeles while recovering from the car accident',
  0.038281523),
 ("west's perfectionism led the college dropout to have its release postponed three times from its initial date in august 2003",
  0.010809121),
 ('it drew little mainstream attention, and the lead single "keep yourself alive", a brian may composition, sold poorly',
  0.008625651),
 ('west meticulously refined the production, adding string arrangements, gospel choirs, improved drum programming and new verses',
  -0.00013764449),
 ('beginning his second effort that fall, west would invest two million dollars and take over a year to craft his second album',
  -0.0071180295),
 ('as a result, certain tracks originally destined for the album were subsequently retracted

In [66]:
get_answer("What areas did Beyonce compete in when she was growing up?")

66503    [czech distinguishes three genders—masculine, feminine, and neuter—and the masculine gender is subdivided into animate and inanimate, with few exceptions, feminine nouns in the nominative case end in -a, -e, or -ost; neuter nouns in -o, -e, or -í, and masculine nouns in a consonant, adjectives agree in gender and animacy (for masculine nouns in the accusative or genitive singular and the nominative plural) with the nouns they modify, the main effect of gender in czech is the difference in noun and adjective declension, but other effects include past-tense verb endings: for example, dělal (he did, or made); dělala (she did, or made) and dělalo (it did, or made)]
58418    [the system displays the what's new screen by default instead of the [games] menu (or [video] menu, if a movie was inserted) when starting up, what's new has four sections: "our pick", "recently played", latest information and new content available in playstation store, there are four kinds of content the what'

[('czech distinguishes three genders—masculine, feminine, and neuter—and the masculine gender is subdivided into animate and inanimate',
  0.05793891),
 ("there are four kinds of content the what's new screen displays and links to, on the sections",
  0.016624594),
 ('adjectives agree in gender and animacy (for masculine nouns in the accusative or genitive singular and the nominative plural) with the nouns they modify',
  0.0049622916),
 ('^9 sub-groups of croats include bunjevci (in bačka), šokci (in slavonia and vojvodina), janjevci (in kosovo), burgenland croats (in austria), bosniaks (in hungary), molise croats (in italy), krashovans (in romania), moravian croats (in the czech republic)',
  -0.0025368186),
 ('with few exceptions, feminine nouns in the nominative case end in -a, -e, or -ost; neuter nouns in -o, -e, or -í, and masculine nouns in a consonant',
  -0.0040099532),
 ('what\'s new has four sections: "our pick", "recently played", latest information and new content availabl